In [ ]:
# image net dataset - ISLVRC2012 local machine practise

In [1]:
class DefaultConfigs(object):

    train_dir = "../train"
    val_dir = '../val'
    model_name = "resnet18"
    weights = "./checkpoints/"
    best_models = weights + "best_model/"

    epochs = 40
    start_epoch = 0
    batch_size = 16
    momentum = 0.9
    lr = 1e-3
    weight_decay = 1e-4
    interval = 10
    workers = 12

    evaluate = False
    pretrained = False
    resume = False

In [2]:
class AverageMeter(object):

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


def accuracy(output, target, topk=(1,)):

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res